<a href="https://colab.research.google.com/github/MoffatKirui/week4ip/blob/main/Moringa_Data_Science_Prep_W4_Independent_Project_2021_03_Moffat_Kirui_PythonNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***ELECTRIC CAR SHARING SERIVCE COMPANY OPERATION'S ANALYSIS***

---
##**1.Importing our Libraries**


In [2]:
#importing pandas library
import pandas as pd
#importing numpy library
import numpy as np

##**2.Loading our dataset**

---




*   We will be loading our dataset from a url



In [3]:
#url with our dataset
url='http://bit.ly/autolib_dataset'
#reading the csv file from the url and creating a dataframe df
df=pd.read_csv(url)




*   Previewing our data frame and accessing its information




In [4]:
#previewing our dataframe
df.head(3)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14


In [5]:
#accessing information about the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

##**3.Data Cleaning**



###  Validity






In [6]:
#dropping irrelevant columns
df.drop(['Address','Displayed comment','ID','Geo point','Cars','Subscription status','Charge Slots','Charging Status','Scheduled at','Slots'], axis=1,inplace=True)
df.head()

,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,Kind,Postal code,Public name,Rental status,Station type,Status,year,month,day,hour,minute
0,0,0,0,Paris,STATION,75015,Paris/Suffren/2,operational,station,ok,2018,4,8,11,43
1,6,0,0,Paris,STATION,75014,Paris/Raymond Losserand/145,operational,station,ok,2018,4,6,7,24
2,3,0,2,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,station,ok,2018,4,3,20,14
3,3,1,0,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,station,ok,2018,4,4,4,37
4,3,0,0,Paris,PARKING,75017,Paris/Porte de Champerret/6,operational,station,ok,2018,4,8,17,23


###Accuracy

In [31]:
#filtering data to fit conditions for our analys
df=df[(df['city']== 'Paris') & (df['rental_status']== 'operational') & (df['status']== 'ok')]
#dropping the columns as the would be similar for all rows
df.drop(['city','rental_status','status'],axis=1,inplace=True)

###Completeness

In [11]:
#checking for null values in the remaining dataframe
df.isnull().any()

Bluecar counter       False
Utilib counter        False
Utilib 1.4 counter    False
City                  False
Kind                  False
Postal code           False
Public name           False
Rental status         False
Station type          False
Status                False
year                  False
month                 False
day                   False
hour                  False
minute                False
dtype: bool

###Consistency

In [32]:
#checking for duplicates
df[df.duplicated()==True]

,bluecar_counter,utilib_counter,utilib_1.4_counter,kind,postal_code,public_name,station_type,DateTime


###Uniformity

In [41]:
#chaining str functions to ensure uniformity in column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df.head(0)

,bluecar_counter,utilib_counter,utilib_1.4_counter,kind,postal_code,public_name,station_type,datetime


##**4.Deriving new attributes**

In [21]:
#creating a new column with all the date and time objects
df['DateTime']=pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])


In [22]:
#dropping the columns as we have them under one column
df.drop(['year', 'month', 'day', 'hour', 'minute'],axis=1,inplace=True)

In [40]:
df.columns

Index(['bluecar_counter', 'utilib_counter', 'utilib_1.4_counter', 'kind',
       'postal_code', 'public_name', 'station_type', 'datetime'],
      dtype='object')

In [35]:
#creating a copy of our dataframe to alter the values of the cars counter to display usage
#+ve number means that a car was returned -ve number means that a car was picked 0 means that nothing happened, i.e no car was picked nor returned
#
#start by arranging values according to station and datetime
dp=df.sort_values(by=['public_name','datetime'],ascending=True,ignore_index=True)
#altering the columns to display usage
dp['bluecar_counter']=dp.groupby('public_name')['bluecar_counter'].diff().fillna(0)
dp['utilib_counter']=dp.groupby('public_name')['utilib_counter'].diff().fillna(0)
dp['utilib_1.4_counter']=dp.groupby('public_name')['utilib_1.4_counter'].diff().fillna(0)


In [43]:
dp.head()

,bluecar_counter,utilib_counter,utilib_1.4_counter,kind,postal_code,public_name,station_type,datetime
0,0.0,0.0,0.0,STATION,75014,Paris/25 AoÃ»t 1944/8,station,2018-04-01 18:40:00
1,3.0,0.0,0.0,STATION,75014,Paris/25 AoÃ»t 1944/8,station,2018-04-02 07:37:00
2,-2.0,0.0,0.0,STATION,75014,Paris/25 AoÃ»t 1944/8,station,2018-04-02 12:49:00
3,1.0,0.0,0.0,STATION,75014,Paris/25 AoÃ»t 1944/8,station,2018-04-04 03:10:00
4,-2.0,0.0,0.0,STATION,75014,Paris/25 AoÃ»t 1944/8,station,2018-04-04 18:48:00


##**5.Analysis**

###Bluecar

-Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018

In [86]:
#filtering only those cars that were being picked
x=dp[dp['bluecar_counter']<0]
#grouping by hour then selecting the one with most counts
x.groupby(x['datetime'].dt.hour)['bluecar_counter'].count().sort_values(ascending=False).head(1)

datetime
21    44
Name: bluecar_counter, dtype: int64

-What is the most popular hour for returning cars?

In [85]:
#filtering cars that were being returned
x=dp[dp['bluecar_counter']>0]
#grouping by hour then selecting the one with most counts
x.groupby(x['datetime'].dt.hour)['bluecar_counter'].count().sort_values(ascending=False).head(1)

datetime
12    47
Name: bluecar_counter, dtype: int64

-What station is the most popular?


---


Overall?





In [53]:
#grouping by station and picking the most popular
dp.groupby('public_name')['bluecar_counter'].count().sort_values(ascending=False).head(1)

public_name
Paris/Porte de Montrouge/8    13
Name: bluecar_counter, dtype: int64

At the most popular picking hour?

In [55]:
#filtering by the most popular hour
x=dp[dp.datetime.dt.hour==21]
#grouping by station and picking the most popular
x.groupby('public_name')['bluecar_counter'].count().sort_values(ascending=False).head(1)

public_name
Paris/Tronchet/19    4
Name: bluecar_counter, dtype: int64

-What postal code is the most popular for picking up Blue cars? Does the most popular station belong to that postal code?


---

overall?

In [84]:
#filtering bluecars that were being picked
x=dp[dp['bluecar_counter']<0]
#selecting the most popular postal code
x.groupby('postal_code')['bluecar_counter'].count().sort_values(ascending=False).head(1)

postal_code
75015    76
Name: bluecar_counter, dtype: int64

In [59]:
#selecting the most popular station for picking cars and its postal code
x.groupby(['public_name','postal_code'])['bluecar_counter'].count().sort_values(ascending=False).head(1)

public_name                 postal_code
Paris/Porte de Montrouge/8  75014          6
Name: bluecar_counter, dtype: int64

At the most popular picking hour?

In [87]:
#filtering cars that were being picked by the most popular hour
y=x[x.datetime.dt.hour==21]
#selecting most popular postal code
y.groupby('postal_code')['bluecar_counter'].count().sort_values(ascending=False).head(1)

postal_code
75016    6
Name: bluecar_counter, dtype: int64

In [61]:
#selecting most popular station and its postal code at the most popular hour
y.groupby(['public_name','postal_code'])['bluecar_counter'].count().sort_values(ascending=False).head(1)

public_name      postal_code
Paris/Mozart/68  75016          2
Name: bluecar_counter, dtype: int64

###Utilib

-Identify the most popular hour of the day for picking up a shared electric car (Utilib) in the city of Paris over the month of April 2018

In [88]:
#filtering only those cars that were being picked
x=dp[dp['utilib_counter']<0]
#grouping by hour then selecting the one with most counts
x.groupby(x['datetime'].dt.hour)['bluecar_counter'].count().sort_values(ascending=False).head(1)

datetime
23    8
Name: bluecar_counter, dtype: int64

What is the most popular hour for returning cars?

In [64]:
#filtering cars that were being picked
x=dp[dp['utilib_counter']>0]
#grouping by hour then selecting the one with most counts
x.groupby(x['datetime'].dt.hour)['bluecar_counter'].count().sort_values(ascending=False).head(1)

datetime
6    8
Name: bluecar_counter, dtype: int64

-What station is the most popular?

Overall?

In [89]:
#grouping by station and picking the most popular
dp.groupby('public_name')['utilib_counter'].count().sort_values(ascending=False).head(1)

public_name
Paris/Porte de Montrouge/8    13
Name: utilib_counter, dtype: int64

At the most popular picking hour?

In [90]:
x=dp[dp.datetime.dt.hour==23]
#grouping by station and picking the most popular
x.groupby('public_name')['bluecar_counter'].count().sort_values(ascending=False).head(1)

public_name
Paris/Adolphe Yvon/6    2
Name: bluecar_counter, dtype: int64

What postal code is the most popular for picking up Blue cars? Does the most popular station belong to that postal code?

Overall?

In [67]:
#filtering cars that were being picked
x=dp[dp['utilib_counter']<0]
#selecting the most popular postal code
x.groupby('postal_code')['bluecar_counter'].count().sort_values(ascending=False).head(1)

postal_code
75017    13
Name: bluecar_counter, dtype: int64

In [68]:
#selecting the most popular station for picking cars and its postal code
x.groupby(['public_name','postal_code'])['bluecar_counter'].count().sort_values(ascending=False).head(1)

public_name                 postal_code
Paris/Porte de Montrouge/8  75014          2
Name: bluecar_counter, dtype: int64

At the most popular picking hour?

In [69]:
#filtering cars that were being picked by the most popular hour
y=x[x.datetime.dt.hour==23]
#selecting most popular postal code
y.groupby('postal_code')['bluecar_counter'].count().sort_values(ascending=False).head(1)

postal_code
75013    2
Name: bluecar_counter, dtype: int64

In [70]:
#selecting most popular station and its postal code at the most popular hour
y.groupby(['public_name','postal_code'])['bluecar_counter'].count().sort_values(ascending=False).head(1)

public_name               postal_code
Paris/Vincent Auriol/143  75013          1
Name: bluecar_counter, dtype: int64

###Utilib_1.4

-Identify the most popular hour of the day for picking up a shared electric car (Utilib) in the city of Paris over the month of April 2018

In [71]:
#filtering only those cars that were being picked
x=dp[dp['utilib_1.4_counter']<0]
#grouping by hour then selecting the one with most counts
x.groupby(x['datetime'].dt.hour)['bluecar_counter'].count().sort_values(ascending=False).head(1)

datetime
18    15
Name: bluecar_counter, dtype: int64

What is the most popular hour for returning cars?

In [72]:
#filtering cars that were being picked
x=dp[dp['utilib_1.4_counter']>0]
#grouping by hour then selecting the one with most counts
x.groupby(x['datetime'].dt.hour)['bluecar_counter'].count().sort_values(ascending=False).head(1)

datetime
21    14
Name: bluecar_counter, dtype: int64

-What station is the most popular?

Overall?

In [91]:
#grouping by station and picking the most popular
dp.groupby('public_name')['utilib_counter'].count().sort_values(ascending=False).head(1)

public_name
Paris/Porte de Montrouge/8    13
Name: utilib_counter, dtype: int64

At the most popular picking hour?

In [92]:
x=dp[dp.datetime.dt.hour==18]
#grouping by station and picking the most popular
x.groupby('public_name')['bluecar_counter'].count().sort_values(ascending=False).head(1)

public_name
Paris/25 AoÃ»t 1944/8    2
Name: bluecar_counter, dtype: int64

What postal code is the most popular for picking up Blue cars? Does the most popular station belong to that postal code?

Overall?

In [73]:
#filtering cars that were being picked
x=dp[dp['utilib_1.4_counter']<0]
#selecting the most popular postal code
x.groupby('postal_code')['bluecar_counter'].count().sort_values(ascending=False).head(1)

postal_code
75017    24
Name: bluecar_counter, dtype: int64

In [74]:
#selecting the most popular station for picking cars and its postal code
x.groupby(['public_name','postal_code'])['bluecar_counter'].count().sort_values(ascending=False).head(1)

public_name       postal_code
Paris/Lourmel/33  75015          3
Name: bluecar_counter, dtype: int64

At the most popular picking hour?

In [75]:
#filtering cars that were being picked by the most popular hour
y=x[x.datetime.dt.hour==18]
#selecting most popular postal code
y.groupby('postal_code')['bluecar_counter'].count().sort_values(ascending=False).head(1)

postal_code
75016    3
Name: bluecar_counter, dtype: int64

In [76]:
#selecting most popular station and its postal code at the most popular hour
y.groupby(['public_name','postal_code'])['bluecar_counter'].count().sort_values(ascending=False).head(1)

public_name         postal_code
Paris/Voltaire/236  75011          1
Name: bluecar_counter, dtype: int64